In [2]:
#!/usr/bin/python3

# Import statements
import cv2
import numpy as np
import os

def main():
    imageNames = os.listdir('test_images/')

    for imageName in imageNames:
        # Read image
        frame = cv2.imread('test_images/'+imageName)
        processImage(frame)
        
    cv2.destroyAllWindows()
    
    
def processImage(imgSource):
    
    # Convert image to grayscale
    imgGray = cv2.cvtColor(imgSource, cv2.COLOR_BGR2GRAY)

    # Smoothen image
    imgBlur = cv2.GaussianBlur(imgGray, (5,5), 3)

    # Get edge image using Canny edge detector
    threshLow = 65
    threshHigh = 200
    imgEdge = cv2.Canny(imgBlur, threshLow, threshHigh)

    # Create mask to select only the lane in front of vehicle
    mask = np.copy(imgEdge) * 0
    vertices = np.array([[(120, imgSource.shape[0]), (430, 340), (550, 340), (910, imgSource.shape[0])]])
    cv2.fillPoly(mask, vertices, (255, 255, 255))
        
    # Apply mask to the imag
    imgMasked = cv2.bitwise_and(imgEdge, mask)

    # Draw lines
    drawLines(imgMasked, imgSource)
    
    
def drawLines(imgMasked, imgSource):
    
    # Get lines from masked edge image using Hough transform
    rho = 2
    theta = np.pi/180
    threshold = 15
    minLineLength = 100
    maxLineGap = 150
    lines = cv2.HoughLinesP(imgMasked, rho, theta, threshold, np.array([]), minLineLength, maxLineGap)
    
    if lines.size:
        slopePos = []; slopeNeg = []; ptsLeft = []; ptsRight = []

        # Calcualte slope
        for line in lines:
            for x1, y1, x2, y2 in line:
                if((x2-x1)!=0 and (y2-y1)!=0):
                    slope = float(y2-y1)/float(x2-x1)
                    angle = np.rad2deg(np.arctan2(y2-y1, x2-x1))
                    
                    if((angle > 25 and angle < 45) or (angle < -25 and angle > -45)):
                        # Right Line
                        if(slope < 0):
                            slopeNeg.append(slope)
                            ptsRight.append([x1, y1])
                        # Left Line
                        else:
                            slopePos.append(slope)
                            ptsLeft.append([x1,y1])
        
        if(len(slopeNeg)!=0 and len(slopePos)!=0):
            #slopeRight = np.mean(slopeNeg)
            #slopeLeft = np.mean(slopePos)
            slopeRight = slopeNeg[0]
            slopeLeft = slopePos[0]

            # Create black image of same size as edge image
            imgLines = np.copy(imgSource) * 0

            yTop = 340

            # Draw Right Line
            intercept = ptsRight[0][1] - slopeRight * ptsRight[0][0]
            xBottom = int( (imgSource.shape[0] - intercept) / slopeRight )
            xTop = int( (yTop - intercept) / slopeRight )
            cv2.line(imgLines, (xTop, yTop), (xBottom, imgSource.shape[0]), (200,50,0), 12)

            # Draw Left Line
            intercept = ptsLeft[0][1] - slopeLeft * ptsLeft[0][0]
            xBottom = int( (imgSource.shape[0] - intercept) / slopeLeft )
            xTop = int( (yTop - intercept) / slopeLeft )
            cv2.line(imgLines, (xTop, yTop), (xBottom, imgSource.shape[0]), (200,50,0), 12)

            # Add to source image
            imgLines = cv2.addWeighted(imgSource, 1.0, imgLines, 0.8, 1.0)

            # Display annotated image
            cv2.imshow('Image Lines', imgLines)
        
        else:
            cv2.imshow('Image Lines', imgSource)
            
    else:
        cv2.imshow('Image Lines', imgSource)
    
    key = cv2.waitKey(2000)


if __name__ == "__main__": 
    main()